In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
da=pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

# Exploring Data

In [ ]:
da.head()


In [ ]:
da.columns

In [ ]:
da.value_counts()

In [ ]:
da.isna().sum()

In [ ]:
da.describe()

### the column 'bmi' has .039% null values (201 0ut of 5110  )

In [ ]:
#correaltion between numerical types of columns
da.corr()

In [ ]:
da_ob=da.select_dtypes(object)

In [ ]:
da.dtypes

In [ ]:
da_int=da.select_dtypes(int)

In [ ]:
for column in da.columns:
    if column in da_int.columns:
        print(column,':',da[str(column)].unique())
    

Here other than id,all other columns represents either patient has disease or not.

In [ ]:
for column in da.columns:
    if column in da_ob.columns:
        print(column,':',da[str(column)].unique())
    

from the correlation matrix it is clear that id has no relation with stroke.So we can avoid it.

In [ ]:
from matplotlib import pyplot as plt

# Histogram of Numerical Columns

In [ ]:
for column in da.columns:
    if column not in da_ob.columns :
        if column not in da_int.columns:
            plt.hist(da[str(column)])
            plt.title(str(column))
            plt.show()

    

In [ ]:
import seaborn as sns

# Categories Vs  Stroke

In [ ]:
for column in da_ob.columns:
    sns.catplot(x=da[str(column)],y=da['stroke'],kind='bar',data=da)

# Now lets convert every data types to numeric


In [ ]:
df=da.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
for c in df.columns:
    if c in da_ob:
        df[c]=label_encoder.fit_transform(df[c])

In [ ]:
df.head()

Now we have to deal with null values in bmi since only .04% values are there.Its okay to avoid those rows

In [ ]:
df = df[df['bmi'].notna()]

In [ ]:
df.isna().sum()

In [ ]:
df=df.drop(['id'],axis=1)

In [ ]:
df.head()

In [ ]:
df.dtypes

Our problem is to check wether a person will have a stroke given values of features as given here.Let us try how logistic regression fit to the data.we use sklearn intially to check perfomance.

In [ ]:
Target=df['stroke']

In [ ]:
features=df.drop(['stroke'],axis=1)

In [ ]:
features

In [ ]:
X_data=np.array(features)
Y=np.array(Target)

In [ ]:
from sklearn import preprocessing

In [ ]:
X=preprocessing.normalize(X_data)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=101)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

Logistic Regression gives almost 95% accuraccy

Now lets try to implement Logistic regression using pytorch

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
class logistic_Regression(torch.nn.Module):
    def __init__(self):
        super(logistic_Regression,self).__init__()
        self.linear = torch.nn.Linear(66,1)
    def forward(self, x):
        y_pred = F.sigmoid(self.linear(x))
        return y_pred
model = logistic_Regression()
        

In [ ]:
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
X_train=torch.tensor(X_train)
y_train=torch.tensor(y_train).reshape(-1,1)

In [ ]:
epoch_num=100
for epoch in range(epoch_num):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(X_train.float())
        # Compute Loss
    loss = criterion(y_pred, y_train.float())
    # Backward pass
    loss.backward()
    optimizer.step()

In [ ]:
y_pred=model(torch.tensor(X_test).float())

y_new=[round(y_pred[i] .detach().numpy()[0])for i in range(len(y_pred))]

In [ ]:
y_new

In [ ]:
torch.tensor(y_test).reshape(-1,1)==y_pred

In [ ]:
sum(y_test==y_new)/len(y_new)

That is the same score as the sckit model